In [2]:
import pandas as pd

#Step 1 Read Data
keywords_df = pd.read_excel("/Users/baymaxshawn/Desktop/review_assessment/Keywords Input.xlsx")
opc_labels_df = pd.read_excel("/Users/baymaxshawn/Desktop/review_assessment/OPC Labels Input.xlsx")

sheet1_df = pd.read_excel("/Users/baymaxshawn/Desktop/review_assessment/opc_review_output.xlsx", sheet_name="1")
sheet2_df = pd.read_excel("/Users/baymaxshawn/Desktop/review_assessment/opc_review_output.xlsx", sheet_name="2")
sheet3_df = pd.read_excel("/Users/baymaxshawn/Desktop/review_assessment/opc_review_output.xlsx", sheet_name="3")

In [5]:
print(keywords_with_opc_labels.columns)

Index(['short_id', 'keyword_text', 'SPA Rank', 'bid', 'OPC label'], dtype='object')


In [18]:
#Step 2

#Check Assign OPC Label 
keywords_with_opc_labels = pd.merge(keywords_df, opc_labels_df, on='keyword_text', how='left')
missing_opc_labels = keywords_with_opc_labels[keywords_with_opc_labels['OPC label'].isnull()]

if not missing_opc_labels.empty:
    error_type = "Missing OPC Label"
    example = missing_opc_labels.iloc[0] 
    print(f"Error Class/Type: {error_type}")
    print(f"Example: {example}/n")

In [33]:
# Check Perform bid swaps 
before_swap_df = sheet1_df[['OPC Rank', 'bid', 'SPA Rank']].copy()

out_of_position_short_ids = sheet1_df.dropna(subset=['SPA Rank'])
out_of_position_short_ids = out_of_position_short_ids[out_of_position_short_ids['OPC Rank'] != out_of_position_short_ids['SPA Rank']]

missing_spa_values = out_of_position_short_ids['SPA Rank'].isnull().sum()
print(f"Number of missing SPA values: {missing_spa_values}")

for i, row in out_of_position_short_ids.iterrows():
    higher_opc_rank = row['OPC Rank']
    higher_bid = row['bid']
    lower_opc_rank = row['SPA Rank']

    matching_rows = before_swap_df[before_swap_df['OPC Rank'] == lower_opc_rank]

    if not matching_rows.empty and not matching_rows['bid'].isnull().all():
        lower_bid = matching_rows['bid'].values[0]
    else:
        lower_bid = None
        
    before_swap_df.loc[before_swap_df['OPC Rank'] == lower_opc_rank, 'bid'] = higher_bid
    before_swap_df.loc[before_swap_df['OPC Rank'] == higher_opc_rank, 'bid'] = lower_bid

if not before_swap_df.equals(sheet1_df[['OPC Rank', 'bid', 'SPA Rank']]):
    error_type = "Bid Swap Error"
    example = before_swap_df.iloc[0]
    print(f"Error Class/Type: {error_type}")
    print(f"Example: {example}\n")



Number of missing SPA values: 0
Error Class/Type: Bid Swap Error
Example: OPC Rank    1.0
bid         0.1
SPA Rank    NaN
Name: 0, dtype: float64



In [24]:
missing_bid_rows = sheet1_df[sheet1_df['bid'].isnull()]
print(missing_bid_rows)


Empty DataFrame
Columns: [short_id, keyword_text, SPA Rank, bid, OPC Label, OPC Rank, bid_rank, temp_new_bid, final_new_bid]
Index: []


In [25]:
#Check Identifying any short IDs 
higher_ranked_short_ids = sheet2_df[sheet2_df['SPA Rank'].isnull() & (sheet2_df['OPC Rank'] > sheet2_df['SPA Rank'])]

for i, row in higher_ranked_short_ids.iterrows():
    max_bid_of_lower_ran_spots = sheet2_df[sheet2_df['SPA Rank'] < row['OPC Rank']]['bid'].max()
    bid_increment = max_bid_of_lower_ran_spots * 0.15
    new_bid = max(bid_increment, row['bid'])

    #qualif
    if new_bid < row['bid']:
        error_type = "Bid Increment Error"
        example = row
        print(f"Error Class/Type: {error_type}")
        print(f"Example: {example}\n")
